<h1 align=center><font size = 5>Incorporating Coordinate</font></h1>

##### Get the table

In [3]:
import pandas as pd
import urllib.request

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)

from bs4 import BeautifulSoup
soup = BeautifulSoup(page, "lxml")
all_tables=soup.find_all("table")
right_table=soup.find('table', class_='wikitable sortable')
postal_codes=[]
boroughs=[]
neighborhoods=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        postal_codes.append(cells[0].find(text=True))
        boroughs.append(cells[1].find(text=True))
        neighborhoods.append(cells[2].find(text=True))


postal_codes = [postal_code.rstrip('\n') for postal_code in postal_codes] 
boroughs = [borough.rstrip('\n') for borough in boroughs] 
neighborhoods = [neighborhood.rstrip('\n') for neighborhood in neighborhoods] 

Toronto_FSAs = pd.DataFrame(postal_codes,columns=['Postal Code'])
Toronto_FSAs['Borough'] = boroughs
Toronto_FSAs['Neighborhood'] = neighborhoods

Toronto_FSAs = Toronto_FSAs[Toronto_FSAs.Borough != 'Not assigned'].reset_index(drop=True)

Toronto_FSAs

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [2]:
import geocoder

def get_geocoder(postal_code):

  lat_lng_coords = None


  while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng

  latitude = lat_lng_coords[0]
  longitude = lat_lng_coords[1]
  
  return latitude, longitude

In [ ]:
Toronto_FSAs['Latitude'], Toronto_FSAs['Longitude'] = Toronto_FSAs['Postal Code'].apply(get_geocoder)

#### This package does not work

##### Read CSV File Instead

In [4]:
coordinate = pd.read_csv("http://cocl.us/Geospatial_data")

In [5]:
Toronto_FSAs_with_coor = Toronto_FSAs

In [7]:
Toronto_FSAs_with_coor.merge(coordinate, on = 'Postal Code')

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
